In [1]:
%load_ext autoreload
%autoreload 2

import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

import pandas as pd
import pickle
import numpy as np
import torch
import matplotlib.pylab as plt
from test_functions import problem_setup
from sim_helpers import gen_rand_X
from scipy.optimize import minimize

plt.rc("axes.spines", top=False, right=False)
colors_palette = plt.rcParams['axes.prop_cycle'].by_key()['color']

problem_strs_dict = {
    "vehiclesafety_5d3d_kumaraswamyproduct": "Vehicle safety (d=5, k=3) \n Product of Kumaraswamy CDFs",
    'dtlz2_8d4d_negl1dist': 'DTLZ2 (d=8, k=4) \n L1 distance',
    'osy_6d8d_piecewiselinear': 'OSY (d=6, k=8) \n Piece-wise linear',
    "carcabdesign_7d9d_piecewiselinear": "Car cab design (d=7, k=9) \n Piece-wise linear", 
    
    "vehiclesafety_5d3d_piecewiselinear": "Vehicle safety (d=5, k=3) \n Piece-wise linear",
    "dtlz2_8d4d_piecewiselinear": "DTLZ2 (d=8, k=4) \n Piece-wise linear",
    'osy_6d8d_sigmodconstraints': 'OSY (d=6, k=8) \n Exp. func. sum with sigmoid constraints',
    "carcabdesign_7d9d_linear": "Car cab design (d=7, k=9) \n Linear",
}

max_Xs = {}
max_utils = {}
for problem_str in problem_strs_dict.keys():
    X_dim, Y_dim, problem, util_type, get_util, Y_bounds, probit_noise = problem_setup(problem_str, dtype=torch.double)    

vehiclesafety_5d3d_kumaraswamyproduct, noisy: False, noise_std: 0
dtlz2_8d4d_negl1dist, noisy: False, noise_std: 0
osy_6d8d_piecewiselinear, noisy: False, noise_std: 0
carcabdesign_7d9d_piecewiselinear, noisy: False, noise_std: 0
vehiclesafety_5d3d_piecewiselinear, noisy: False, noise_std: 0
dtlz2_8d4d_piecewiselinear, noisy: False, noise_std: 0
osy_6d8d_sigmodconstraints, noisy: False, noise_std: 0
carcabdesign_7d9d_linear, noisy: False, noise_std: 0


In [2]:
std_norm = torch.distributions.normal.Normal(0, 1)

def estimate_error_rate(x, utils, true_comps):
    choose_0_prob = std_norm.cdf((utils[:, 0] - utils[:, 1]) / x)
    correct_prob = torch.cat((choose_0_prob[true_comps], 1 - choose_0_prob[~true_comps]))
    error_rate = 1 - correct_prob.mean()
    return error_rate.item()

def error_rate_loss(x, utils, true_comps):
    return abs(estimate_error_rate(x, utils, true_comps) - target_error_size)

probit_noise_dict = {}

top1_perc_avg_util = {}

for problem_str in problem_strs_dict.keys():
    step_size = 0.001
    
    # get top random points
    top_proportion = 0.1
    target_error_size = 0.1
    n_samples = int(10000 / top_proportion)
    X_dim, Y_dim, problem, util_type, get_util, Y_bounds, probit_noise = problem_setup(problem_str)
    X = gen_rand_X(n_samples, problem)
    Y = problem(X)
    utils = get_util(Y)
    # take top 10 percent
    utils = utils.sort().values[-int(n_samples * top_proportion):]
     # reshuffle
    utils = utils[torch.randperm(utils.shape[0])]
    utils = utils.reshape(-1, 2)
    top1_perc_avg_util[problem_str] = utils.mean().item()
    
    util_range = (utils.max() - utils.min()).item()

    # estimate probit error
    true_comps = (utils[:, 0] >= utils[:, 1])

    res = minimize(error_rate_loss, x0=0.01, args=(utils, true_comps))
    probit_noise = res.x[0]
    probit_noise = round(probit_noise, 4)
    
    error_rate = estimate_error_rate(probit_noise, utils, true_comps)
    
    probit_noise_dict[problem_str] = probit_noise
    print(f"{problem_str} error rate: {error_rate:.3f} with probit_noise={probit_noise:.5f}, util_range={util_range:.2f}")
    print()

vehiclesafety_5d3d_kumaraswamyproduct, noisy: False, noise_std: 0
vehiclesafety_5d3d_kumaraswamyproduct error rate: 0.100 with probit_noise=0.01980, util_range=0.30

dtlz2_8d4d_negl1dist, noisy: False, noise_std: 0
dtlz2_8d4d_negl1dist error rate: 0.100 with probit_noise=0.04560, util_range=0.73

osy_6d8d_piecewiselinear, noisy: False, noise_std: 0
osy_6d8d_piecewiselinear error rate: 0.100 with probit_noise=2.41970, util_range=20.06

carcabdesign_7d9d_piecewiselinear, noisy: False, noise_std: 0
carcabdesign_7d9d_piecewiselinear error rate: 0.100 with probit_noise=0.11010, util_range=1.28

vehiclesafety_5d3d_piecewiselinear, noisy: False, noise_std: 0
vehiclesafety_5d3d_piecewiselinear error rate: 0.100 with probit_noise=0.15440, util_range=2.46

dtlz2_8d4d_piecewiselinear, noisy: False, noise_std: 0
dtlz2_8d4d_piecewiselinear error rate: 0.100 with probit_noise=0.18960, util_range=2.86

osy_6d8d_sigmodconstraints, noisy: False, noise_std: 0
osy_6d8d_sigmodconstraints error rate: 0.100